In [8]:
# !pip install dspy-ai cloudpickle matplotlib

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Import relevant packages
import dspy
from dspy.signatures.signature import signature_to_template
# Setting up LLM models
PROJECT_ID = os.environ.get("PROJECT_ID")
MODEL_ID = "gemini-1.5-flash-001"

flash = dspy.GoogleVertexAI(
    model=MODEL_ID, 
    max_tokens=1000,
    temperature=0.05, 
    project=PROJECT_ID,
)

# Note: different models can be set for prompt model and task model
# - prompt model : rm (retrieval model) 
# - task model : lm (language model)
dspy.settings.configure(lm=flash, rm=flash)

# Fixed themes for the prompt
themes = """
    Technical Learning Resources,
    Clarity of Requirements,
    Time Commitment / Workload,
    Showcase / Presentation Format,
    Communication and Information Sharing,
    Team Formation and Dynamics,
    Relevance of Training Content,
    Application to Business / Use Cases,
    Accessibility and Inclusion,
    Incentives and Recognition,
    Post-Hackathon Follow-up,
    Support from Leadership / Mentors,
"""

1. Define Task

In [8]:
# Setting up signature for DSPy
class GenerateTheme(dspy.Signature):
    """Classify user feedback from hackathon to one single theme"""
    comments = dspy.InputField(desc="user feedback from hackathon")
    themes = dspy.OutputField(desc=f"only pick one of the following choices: {themes}")


class FollowupQuery(dspy.Signature):
    """Generate a query which is conducive to classifying the comment"""

    context = dspy.InputField(desc=f"contains relevant facts to classify comments from hackathon to {themes}")
    comments = dspy.InputField(desc="user feedback from hackathon")
    search_query = dspy.OutputField(desc="Judge if the context is adequate to classify user comments, if not adequate or if it is blank, generate a search query that would help you classify the comments")
    

In [9]:
## Vanilla
class Vanilla(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(GenerateTheme)
    
    def forward(self, comments):
        answer = self.generate_answer(comments=comments)
        theme = answer.themes.split("Themes: ",1)[1].strip()
        return dspy.Prediction(answer=theme)
    
vanilla = Vanilla()

## COT
class COT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateTheme)
    
    def forward(self, comments):
        answer = self.generate_answer(comments=comments)
        return dspy.Prediction(answer=answer.themes)
    
cot = COT()

## ReAct
class ReAct(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ReAct(GenerateTheme)
    
    def forward(self, comments):
        answer = self.generate_answer(comments=comments)
        return dspy.Prediction(answer=answer.themes)
    
react = ReAct()


## BasicMultiHop
class BasicMultiHop(dspy.Module):
    def __init__(self, num_passages=3):
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought(FollowupQuery)
        self.generate_answer = dspy.ChainOfThought(GenerateTheme)

    def forward(self, comments):
        context = []

        for hop in range(2):
            query = self.generate_query(context=context, comments=comments).search_query
            context += self.retrieve(query).passages

        answer = self.generate_answer(context=context, comments=comments)
        return dspy.Prediction(answer=answer.themes)
    
multihop = BasicMultiHop(num_passages=3)


In [ ]:
# Test a question
rag = cot
pred = rag.forward(
    "The topic and content was very interesting and the opportunity to connect with technical and commercial mentors was great! In the future, it would be great to see either more structure to the hackathon, or more communication about the structure of the hackathon."
)
pred
# Check prompt history
# flash.inspect_history(n=1)

2) Import training and testing data

In [2]:
# Create training and testing dataset
import pandas as pd
from dspy.datasets import DataLoader

# Convert csv to dataframe
data = pd.read_csv("./data.csv")
dataset = data[["comments", "answer"]]
dataset = dataset[
    dataset.comments.notna() & dataset.answer.notna()
]
# dataset = dataset.head(5) # limit dataset for testing

# Load data into the desired format for DSPy
dl = DataLoader()
dspy_dataset = dl.from_pandas(dataset, fields=("comments", "answer"), input_keys=['comments'])
splits = dl.train_test_split(dspy_dataset, train_size=0.8) # `dataset` is a List of dspy.Example
train_dataset = splits['train']
test_dataset = splits['test']

dataset.describe()

3. Define permutations for our model candidates

In [5]:
from dspy.teleprompt import LabeledFewShot, BootstrapFewShot

metric = dspy.evaluate.metrics.answer_exact_match

modules = {
    'vanilla': vanilla,
    'cot': cot,
    'react': react,
    'multihop': multihop,
}

optimizers = {
    'none': None,
    'labeled_few_shot': LabeledFewShot(),
    'bootstrap_few_shot': BootstrapFewShot(metric=metric, max_errors=20),
    # 'bootstrap_few_shot_random_search': BootstrapFewShotWithRandomSearch(
    #                                         max_bootstrapped_demos=8,
    #                                         max_labeled_demos=8,
    #                                         num_candidate_programs=10,
    #                                         num_threads=8,
    #                                         metric=metric,
    #                                         teacher_settings=dict(lm=flash)
    #                                     )    
}

Here we define a helper class to facilitate the evaluation

In [6]:
from dspy.evaluate.evaluate import Evaluate
import pandas as pd

NUM_THREADS = 4
class ModelSelection():

    # Compile our models
    def __init__(self, modules, optimizers, metric, trainset):
        self.models = []
        self.metric = metric
        
        for module_name, module in modules.items():
            print(f'Compiling models for {module_name}...')
            models_for_a_program = {'module_name': module_name, 'optimizers': []}

            for optimizer_name, optimizer in optimizers.items():
                print(f'...{optimizer_name}')
                if optimizer is None:
                    compiled_model = module
                else:
                    compiled_model = optimizer.compile(student=module, trainset=trainset)

                optimizer = {
                        'name': optimizer_name,
                        'compiled_model': compiled_model
                }

                models_for_a_program['optimizers'].append(optimizer)

            self.models.append(models_for_a_program)

    # Evaluate our models against the testset. After evaluation, we will have a matrix of models and their scores under the evaluation_matrix attribute
    def evaluate(self, testset):
        evaluator = Evaluate(devset=testset, metric=self.metric, num_threads=3, return_outputs=True)
        for module in self.models:
            print(f"""Evaluating models for {module['module_name']}...""")
            for optimizer in module['optimizers']:
                compiled_model = optimizer['compiled_model']
                evaluation_score, outputs = evaluator(compiled_model)
                optimizer['score'] = evaluation_score

        # read dict into a dataframe
        df = pd.DataFrame(self.models)

        # unnest optimizers column
        df = df.explode('optimizers')

        # extract name/score column from optimizers
        df['optimizer'] = df['optimizers'].apply(lambda x: x['name'])
        df['score'] = df['optimizers'].apply(lambda x: x['score'])

        df.drop(columns=['optimizers'], inplace=True)
        self.evaluation_matrix = df

    # Raise a question against the compiled model
    def question_for_model(self, module_name, optimizer_name, comments):
        for model in self.models:
            if model['module_name'] == module_name:
                for s in model['optimizers']:
                    if s['name'] == optimizer_name:
                        return s['compiled_model'](comments=comments)

4. Train and evaluate the different models

In [ ]:
# Compile the models
ms = ModelSelection(modules=modules, optimizers=optimizers, metric=metric, trainset=train_dataset)

# Evaluate them
ms.evaluate(testset=test_dataset)
print("Done!")

In [ ]:
ms.evaluation_matrix

In [ ]:
# Test with compiled model
pred = ms.question_for_model("cot", "labeled_few_shot", "Slightly fewer final presentations, and each presenting team presenting for slightly longer")
pred

In [ ]:
flash.inspect_history(n=1)

(Optional) Install Phoenix to inspect history

In [12]:
# !pip install "regex~=2023.10.3" arize-phoenix openinference-instrumentation-vertexai vertexai

In [ ]:
# Phoenix by default uses the 6006 port for the UI. 
# If you have a port conflict, you can close the port by uncommenting the following code

import phoenix as px
phoenix_session = px.launch_app()


In [ ]:
from phoenix.otel import register
from openinference.instrumentation.vertexai import VertexAIInstrumentor
tracer_provider = register(
  project_name="default", # Default is 'default'
)  
VertexAIInstrumentor().instrument(tracer_provider=tracer_provider)


In [ ]:
# Used to kill phoenix
import psutil

def close_port(port):
    for conn in psutil.net_connections(kind='inet'):
        if conn.laddr.port == port:
            print(f"Closing port {port} by terminating PID {conn.pid}")
            process = psutil.Process(conn.pid)
            process.terminate()

close_port(6006)